In [6]:
import pandas as pd
# Task 01
# 1.a: load data frame from file, store into df
df = pd.read_csv('KNNAlgorithmDataset.csv')
#print(df)

#1.c: shuffle the samples
df = df.sample(frac=1)  #??? q01: if re-indexing needed? 
# frac = [0,1], incase frac = 0.5, shuffle 50% of the rows
#print(df)

# convert data frame into array
data_array = df.values[:, 1:]  #unselect 1st column / label y

# 1.b: set column 'diagnosis' as target variable y / the label
y = df["diagnosis"].values
#print(y) 

[[1.487e+01 1.171e+01 1.048e+01 ... 1.614e+01 1.232e+01 1.386e+01]
 [2.021e+01 1.719e+01 1.498e+01 ... 1.486e+01 1.239e+01 1.693e+01]
 [9.612e+01 7.468e+01 6.749e+01 ... 1.043e+02 7.885e+01 9.096e+01]
 ...
 [1.017e-01 1.099e-01 9.310e-02 ... 1.129e-01 9.391e-02 1.654e-01]
 [2.369e-01 2.572e-01 3.020e-01 ... 2.778e-01 2.827e-01 3.630e-01]
 [6.599e-02 7.097e-02 9.646e-02 ... 7.012e-02 6.771e-02 1.059e-01]]


In [10]:
# Task 02: seperate data into training(70%), validation(20%) and test(10%) sets by slicing
da_len = len(data_array)   #number of rows / length of data_array
training_set = data_array[:int(da_len * 0.7)]
#print(training_set)
validation_set = data_array[int(da_len * 0.7): int(da_len * 0.9)]
test_set = data_array[int(da_len* 0.9):]
#print(len(training_set) + len(validation_set) + len(test_set) == da_len) # the whole data array included in the 3 sets: True 

In [11]:
# 3.c distance algorithms with input of two 1D-Vectors, return distance (real number) of 2 vectors(x1,...,xn)
import numpy as np
# Manhattan distance
def manhattan_distance(vector01, vector02):
    # raise error if data points are not of the same length
    if (len(vector01) != len(vector02)):
        raise ValueError("Unequal length of inputed data points")
    # return sum(\xi_m - xj_m\)
    return np.sum(np.abs(vector01 - vector02))    # np.abs(): absolute value of each elements
    # a variante: use np.fabs(v1, v2), return np.sum(np.fabs(vector01 - vector02))

# Euclidean distance
def euclidean_distance(vector01, vector02):
    if (len(vector01) != len(vector02)):
        raise ValueError("Unequal length of inputed data points")
    # use np library: calculate the norm of the new vector (difference of two old vectors)
    return np.linalg.norm(vector01 - vector02)

# Chebyshev distance
def chebyshev_distance(vector01, vector02):
    if (len(vector01) != len(vector02)):
        raise ValueError("Unequal length of inputed data points")
    # return max(\xi_01 - xi_02\)
    absolute_array = [np.abs(vector01[i] - vector02[i]) for i in range(len(vector01))]
    return max(absolute_array)


# test algo with data: 
#t_p01 = training_set[0]
#t_p02 = training_set[1]
#t_short01 = [1.1,2,4.43,8]
#t_short02 = [3,4.7,6,9.69]

#print(manhattan_distance(t_p01, t_p02))    # test (compared with result by online calculator): True
#print(euclidean_distance(t_p01, t_p02))    # definition from numpy library
#print(chebyshev_distance(t_short01, t_short02))    # test: True

In [7]:
# Task 3: implement the kNN algorithm
# accroding to 2.2 kNN algo is capsuled in a kNN predictor
class kNN_predictor:
    #constructor: define attribiutes
    def __init__(self, k, distance_func):    #q: if the k and dist-func is given as param in constructor or later in predict func?
        self.trained = False   #default: not trained
        self.k = k  # k neighbours
        self.d = distance_func
        # X(data set) and Y(classes / target values) will be provided in fit() as parameters
        self.X = None
        self.Y = None
        # something about the classification / regression calculated based on data from training set
        
    # functions
    def fit(self, training_X, training_Y):
        #feed the model with training data
        self.X = training_X
        self.Y = training_Y
        self.trained = True   #set trained

    def predict(self):
        #predict classification or regression result for new data.
        # check if predictor is trained
        if self.trained == False:
            raise("Predictor is not trained! Train it first by calling fit()!")
            
        # TODO: call normalize() to normalize the data
        
        # TODO: calculate the distance with all the neighbours
        
        # TODO: choose the k closest neighbours
        
        # TODO: predict / categorize the data point with 0 or 1
        
        return None

    def confusionMatrix(self, X, Y): 
        #compute confusion matrix, input: X (user input)
        '''
        if not self.Y:
            raise error?
        # and validate X and Y shape
        # should be Y given or taken from fit?
            '''
        X = np.array(X)
        Y = np.array(Y)
        pY = np.array([0,1,0,1])        #np.array(predict(X))------------Placeholder

        size = len(X)
        # calculate entries of the confusion matrix by counting entries where the corresponding condition is given and make it relative to the whole set
        TP = len(np.where((Y == 1) & (pY == 1))[0])/size
        TN = len(np.where((Y == 0) & (pY == 0))[0])/size
        FP = len(np.where((Y == 0) & (pY == 1))[0])/size
        FN = len(np.where((Y == 1) & (pY == 0))[0])/size
        return [[TP, FN], [FP, TN]]
  

In [85]:
# task 3.2: check if user inputs are valid
def check_predictor_input(k, d):
    valid_dist_algos = ['manhattan', 'euclidean', 'chebyshev']
    # there are k training set rows / neighbours for the new data point
    if k > len(training_set):
        raise ValueError(f"Given k too large! Give a number not larger than {len(training_set)}")
    if d.lower() not in valid_dist_algos:
        raise ValueError(f"Given algorithm is not valid, please choose from {valid_dist_algos}!")
        
#test: passed
#check_predictor_input(100, 'manha')
#check_predictor_input(400, 'chebyshev')

In [78]:
# task 3.3: make an API, need to be improved
class simple_API:
    def __init__(self, k, d):
        # check if k and d are valid
        check_predictor_input(k, d)
        self.k = k
        self.d = d
        
        # create predictor with inputs
        self.predictor = kNN_predictor(k, d)
        
        # train the predictor with training data 
        x = training_set
        y = y[:len(training_set)]   #slice the labels for training set
        # call predictor.fit() with training data
        self.predictor.fit(x, y)
   
    #predict new data
    def predict(self, newdata):
        return self.predictor.predict(#something)
            
    #a func make graphic?

In [ ]:
# Task 4: test the data with user inputs

In [ ]:
# Task 5: visualization